In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
import sys
sys.path.append('../')
sys.path.append('../../')
from utils import get_config
from gis_resources import san_diego_county_zips
import os
from arcgis.features import GeoAccessor

In [2]:
pd.set_option('display.max_colwidth', None)


In [3]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=1DW2dpwAAKJDFZbKaObn2MT5BZRIaP&allow_verification=false
Enter code obtained on signing in using SAML: ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [8]:
%%time
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_business')

        # display the PostgreSQL database server version
ca_business_result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

CPU times: user 24.4 s, sys: 15.7 s, total: 40.1 s
Wall time: 2min 1s


In [10]:
ca_business_result_list = [(row[1],row[4],row[5],row[7],row[8],row[9]) for row in ca_business_result]
ca_business_result_list[1]

('MoneyGram',
 Decimal('38.516515'),
 Decimal('-121.468739'),
 95824,
 ['Money transfer service'],
 'Sacramento')

In [17]:
# Create a DataFrame
dataFrame = pd.DataFrame(ca_business_result_list,
              columns=("business_name","latitude", "longitude","zip", "categories","city")
              );

In [18]:
# since some of the zip are not present in ca_business table
dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [19]:

dataFrame = dataFrame.astype({"zip": int})
#dataFrame = dataFrame.astype({"zip": str})

In [20]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()
#df[df['col1'].isin(['a', 'c', 'h'])]
#san_diego_county_zips_lst[:10]

In [21]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips_lst)]
dataFrame

,business_name,latitude,longitude,zip,categories,city
7,BJ's Restaurant & Brewhouse,32.7800505,-117.0101371,91942,"[Restaurant, Bar, Brewery, Sports bar]",La Mesa
26,Stars and Stripes Taco Shop,32.877196999999995,-117.186121,92121,"[Mexican restaurant, Fast food restaurant]",San Diego
41,Affordable Jet Ski Rentals,32.8016179,-117.21428949999999,92109,[Water sports equipment rental service],San Diego
57,CVS Pharmacy,32.7554296,-117.2220528,92110,[Pharmacy],San Diego
60,Emergency Room | Kaiser Permanente San Diego Medical Center,32.829720699999996,-117.1252981,92123,[Emergency room],San Diego
...,...,...,...,...,...,...
515865,Ross Dress for Less,32.7910009,-117.0222148,91942,"[Clothing store, Department store, Men's clothing store, Shoe store, Women's clothing store]",La Mesa
515869,Altman Plants,33.2686002,-117.1141689,92026,[Plant nursery],Escondido
515886,Irina's Bar and Grill,33.2180519,-117.34039469999999,92058,"[Restaurant, American restaurant, Bar & grill, Family restaurant, German restaurant, Sports bar]",Oceanside
515897,Mr Pho,33.216423999999996,-117.27185159999999,92056,"[Vietnamese restaurant, Restaurant]",Oceanside


In [22]:
#ca_business_sgdf.head(5)
dataFrame['categories'].fillna('',inplace=True)
dataFrame['categories']=dataFrame.categories.apply(lambda x: ','.join(x))

In [24]:
gdf = geopandas.GeoDataFrame(
    dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [25]:
ca_business_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [26]:
ca_business_sgdf

,business_name,latitude,longitude,zip,categories,city,SHAPE
7,BJ's Restaurant & Brewhouse,32.7800505,-117.0101371,91942,"Restaurant,Bar,Brewery,Sports bar",La Mesa,"{""x"": -117.0101371, ""y"": 32.7800505, ""spatialReference"": {""wkid"": 4326}}"
26,Stars and Stripes Taco Shop,32.877196999999995,-117.186121,92121,"Mexican restaurant,Fast food restaurant",San Diego,"{""x"": -117.186121, ""y"": 32.877196999999995, ""spatialReference"": {""wkid"": 4326}}"
41,Affordable Jet Ski Rentals,32.8016179,-117.21428949999999,92109,Water sports equipment rental service,San Diego,"{""x"": -117.21428949999999, ""y"": 32.8016179, ""spatialReference"": {""wkid"": 4326}}"
57,CVS Pharmacy,32.7554296,-117.2220528,92110,Pharmacy,San Diego,"{""x"": -117.2220528, ""y"": 32.7554296, ""spatialReference"": {""wkid"": 4326}}"
60,Emergency Room | Kaiser Permanente San Diego Medical Center,32.829720699999996,-117.1252981,92123,Emergency room,San Diego,"{""x"": -117.1252981, ""y"": 32.829720699999996, ""spatialReference"": {""wkid"": 4326}}"
...,...,...,...,...,...,...,...
515865,Ross Dress for Less,32.7910009,-117.0222148,91942,"Clothing store,Department store,Men's clothing store,Shoe store,Women's clothing store",La Mesa,"{""x"": -117.0222148, ""y"": 32.7910009, ""spatialReference"": {""wkid"": 4326}}"
515869,Altman Plants,33.2686002,-117.1141689,92026,Plant nursery,Escondido,"{""x"": -117.1141689, ""y"": 33.2686002, ""spatialReference"": {""wkid"": 4326}}"
515886,Irina's Bar and Grill,33.2180519,-117.34039469999999,92058,"Restaurant,American restaurant,Bar & grill,Family restaurant,German restaurant,Sports bar",Oceanside,"{""x"": -117.34039469999999, ""y"": 33.2180519, ""spatialReference"": {""wkid"": 4326}}"
515897,Mr Pho,33.216423999999996,-117.27185159999999,92056,"Vietnamese restaurant,Restaurant",Oceanside,"{""x"": -117.27185159999999, ""y"": 33.216423999999996, ""spatialReference"": {""wkid"": 4326}}"


In [48]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
my_new_featurelayer = ca_business_sgdf.spatial.to_featurelayer(title="San Diego County Businesses", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyBusinesses'])

CPU times: user 35.3 s, sys: 2.21 s, total: 37.5 s
Wall time: 1min 21s


In [28]:
#ca_business_sgdf